relevant values to change are image, lower_bound, upper_bound, erosion_iterations, and connectivity

In [1]:
import cv2
import numpy as np
import math
def change_percent(original :list = [243,230,227] , percent :float = 0.1) -> list:
    return [math.ceil(x*(1+percent)) for x in original]
# Load the image


#### This is the Image Path ####
image = cv2.imread('images/petri-dish3.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for display

percent = 0.30 # Change percent for your liking to increase teh precision of the mask
##169,155,128
# Define the lower and upper bounds for white color in RGB

#### Enter the RGB Values of the average blob color here #### ([r,g,b])
lower_bound = np.array(change_percent([100,100,100],percent=-percent))
upper_bound = np.array(change_percent([243,230,227],percent=percent))

print(lower_bound, upper_bound)
# Create a mask that identifies the white areas in the image
mask = cv2.inRange(image, lower_bound, upper_bound)


[70 70 70] [316 299 296]


In [2]:
%%script true # Skip this Cell


# Blur the image for better edge detection
img_blur = cv2.GaussianBlur(image,(3,3), sigmaX=0, sigmaY=0)
# Convert BGR image to LAB color space
lab = cv2.cvtColor(img_blur, cv2.COLOR_BGR2LAB)
# Split the LAB image into L, A, and B channels
l_channel, a_channel, b_channel = cv2.split(lab)
# Enhance contrast in the L channel using histogram equalization
l_channel_eq = cv2.equalizeHist(l_channel)
# Merge the enhanced L channel with the original A and B channels
lab_eq = cv2.merge((l_channel_eq, a_channel, b_channel))

# Convert LAB image with enhanced L channel back to BGR
contrast = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

cv2.namedWindow('contrast', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('contrast', 800, 800)
# (Optional) Display the mask and the original image with the blobs highlighted
cv2.imshow('contrast', contrast)

In [3]:
%%script true # Skip this Cell

# Apply Canny edge detection
edges = cv2.Canny(image, 50, 150)
#edges = cv2.dilate(edges, kernel, iterations=1)

cv2.namedWindow('edges', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('edges', 800, 800)
# (Optional) Display the mask and the original image with the blobs highlighted
cv2.imshow('edges', edges)

In [4]:
# Apply morphological operations to remove noise
kernel = np.ones((3, 3), np.uint8)

erosion_iterations = 4 # Play Around with this value. the higher the better. but not too high. depending on image size somewhere between 4 and 7 worked for me but i think between 2 and 8 is feasable
dilation_iterations = 0 # keeping this at zero worked best for me

# Erode the mask to remove small white noise
eroded_mask = cv2.erode(mask, kernel, iterations=erosion_iterations)

# Dilate the mask to restore the original size of white areas
dilated_mask = cv2.dilate(eroded_mask, kernel, iterations=dilation_iterations)

cv2.namedWindow('White Areas Mask', cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow('White Areas Mask', 800, 800)
# (Optional) Display the mask and the original image with the blobs highlighted
cv2.imshow('White Areas Mask', dilated_mask)

In [5]:
# Find connected components in the mask
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dilated_mask, connectivity=4) # connectivity either 4 or 8 (4 is more precise for our usecase)

# Subtract one for the background
num_blobs = num_labels - 1

window_title = f'Number of white blobs: {num_blobs}'
print(window_title)

cv2.namedWindow(window_title, cv2.WINDOW_KEEPRATIO)

rect_color = (0, 255, 0) # green

for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    cv2.rectangle(image, (x, y), (x + w, y + h), rect_color , 2)

cv2.imshow(window_title, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
cv2.resizeWindow(window_title, 800, 800)
cv2.resizeWindow('White Areas Mask', 800, 800)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of white blobs: 730
